In [39]:
import torch

def downsample_mask(padding_mask):
    b, t = padding_mask.shape
    return padding_mask[:, ::2]  # Take every second element along the time dimension

# Example
padding_mask = torch.tensor([[1, 1, 0, 0], [1, 0, 1, 0]])  # Shape (2, 4)
new_mask = downsample_mask(padding_mask)
print(new_mask)  # Shape (2, 2)


tensor([[1, 0],
        [1, 1]])


In [44]:
padding_mask, padding_mask.shape

(tensor([[1, 1, 0, 0],
         [1, 0, 1, 0]]),
 torch.Size([2, 4]))

In [46]:
padding_mask[:,1:]

tensor([[1, 0, 0],
        [0, 1, 0]])